In [ ]:
using GuSTO
using AstrobeeRobot

In [ ]:
robot = Freeflyer()
model = FreeflyerSE2()

In [ ]:
env = Table(:stanford);
centers = Vector()
push!(centers, [0.460; 0.315; 0.0])
push!(centers, [0.201; 1.085; 0.0])
push!(centers, [0.540; 2.020; 0.0])
push!(centers, [1.374; 0.196; 0.0])
push!(centers, [1.063; 1.354; 0.0])
push!(centers, [1.265; 2.322; 0.0])
push!(centers, [2.221; 0.548; 0.0])
push!(centers, [2.077; 1.443; 0.0])
push!(centers, [3.098; 1.186; 0.0])
push!(centers, [2.837; 2.064; 0.0])

widths = [0.27; 0.27; 0.127]
inflation = 0.05*ones(3)

for idx in 1:length(centers)
    push!(env.obstacle_set, HyperRectangle(Vec3f0(centers[idx]-0.5*widths-inflation+[0.;0.;0.5*widths[1]]), Vec3f0(widths+2*inflation)))
end

N = 70
x_init = [0.2; 2.4; 0; 0; 0; 0]
x_goal = [3.; 0.5; 0; 0.05; -0.05; 0]

tf_guess = 130.

PD = ProblemDefinition(robot, model, env, x_init, x_goal)

TOP = TrajectoryOptimizationProblem(PD, N, tf_guess, fixed_final_time=true)
TOSgusto = TrajectoryOptimizationSolution(TOP)

SCPP = solve_SCP!(TOSgusto, TOP, solve_gusto_cvx!, init_traj_straightline, "Gurobi", OutputFlag=0);

In [ ]:
@show TOSgusto.SCPS.converged
@show TOSgusto.SCPS.iterations
@show TOSgusto.SCPS.total_time
@show TOSgusto.SCPS.accept_solution
@show TOSgusto.SCPS.prob_status
@show TOSgusto.SCPS.convergence_measure
@show TOSgusto.SCPS.param.alg.omega_vec
@show TOSgusto.SCPS.param.alg.Delta_vec
;

In [ ]:
#### Code for animating free-flyer trajectories
vis = Visualizer()
delete!(vis)

vis[:goal]
setobject!(vis[:goal][:goal], 
    Object(HyperSphere(Point3(x_goal[1:2]..., robot.r), 0.1),
        MeshBasicMaterial(color=RGBA(0,1.0,0.,0.3))))

vis[:table]
table_dim = env.worldAABBmax-env.worldAABBmin
setobject!(vis[:table][:table], 
        Object(HyperRectangle(env.worldAABBmin..., table_dim...),
            MeshBasicMaterial(color=RGBA(0,1.0,0.,0.3))))

vis[:obstacles]
for (idx,obs) in enumerate(env.obstacle_set)
    setobject!(vis[:obstacles][Symbol(string("obs",idx))], 
        Object(obs,MeshBasicMaterial(color=RGBA(1.0,0.,0.,0.3))))
end


vis[:robot]
setobject!(vis[:robot][:base],
    Object(Cylinder(Point3f0(0,0,0.),Point3f0(0.,0.,0.5),Float32(robot.r)),MeshBasicMaterial(color=RGBA(0,0.,1.,0.7))))

up = [0;0;1.]
q0 = vec2quat(up,x_init[3])
Qs = Vector{Vector{Float64}}(0)

speed_factor = 5

for k in 1:speed_factor:N
    q = [quat_inv(quat_multiply(mrp2quat([0.; 0.; tan(TOSgusto.SCPS.traj.X[3,k]/4)]), q0)); [TOSgusto.SCPS.traj.X[1:2,k]; robot.r]]
    push!(Qs,q)
end

anim = Animation()
for k in 1:N
    atframe(anim, vis, 24*k) do frame
        settransform!(frame[:robot], Translation(TOSgusto.traj.X[1,k], 
            TOSgusto.traj.X[2,k], TOSgusto.traj.X[3,k]))
    end
end
setanimation!(vis, anim)
    
plot_in_cell = true
plot_in_cell ? IJuliaCell(vis) : open(vis)